# Проект по защите данных.

Нам нужно защитить данные клиентов страховой компании «Хоть потоп». Требуется разработать такой метод преобразования данных, чтобы по ним было сложно восстановить персональную информацию.

Нужно защитить данные, чтобы при преобразовании качество моделей машинного обучения не ухудшилось. Подбирать наилучшую модель не требуется.

## Загрузим и изучим данные.

In [1]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score

insurance_df = pd.read_csv('insurance.csv')

print(insurance_df.info())
insurance_df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 5 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Пол                5000 non-null   int64  
 1   Возраст            5000 non-null   float64
 2   Зарплата           5000 non-null   float64
 3   Члены семьи        5000 non-null   int64  
 4   Страховые выплаты  5000 non-null   int64  
dtypes: float64(2), int64(3)
memory usage: 195.4 KB
None


,Пол,Возраст,Зарплата,Члены семьи,Страховые выплаты
0,1,41.0,49600.0,1,0
1,0,46.0,38000.0,1,1
2,0,29.0,21000.0,0,0
3,0,21.0,41700.0,2,0
4,1,28.0,26100.0,0,0


**Признаки**: пол, возраст и зарплата застрахованного, количество членов его семьи.

**Целевой признак**: количество страховых выплат клиенту за последние 5 лет.

Пропуски в данных отсутствуют. Все переменные количественного типа, так что необходимости в кодировки нет. Поскольку качество модели улучшать нам не нужно (нужно лишь посмотреть не изменится ли оно), то нормализацию производить не будем.

<font color='blue'> 
### <u>КОММЕНТАРИЙ РЕВЬЮЕРА</u>
</font>
<font color='green'>
лёгкое аналитическое начало)<br>
<font color='orange'>
ВАЖНО!!! по использованию корреляций<br>
0) Этот инструмент - исключительно для данных в количественной шкале!!!!<br>
Т.е. Страховы выплаты и Вораст - корреляцию не посмотреть. Т.к. Одна величина - номинальная (в виде 0 и 1, т.е. да/нет можно сказать), другая количественная. Поэтому для таких ситуаций подойдёт другая техника - сравнение групп (из темы "проверка гипотез" спринт 3 и 4).<br><br>    

## Как умножение матрицы признаков на обратимую матрицу повлияет на предсказания модели?

Вектор предсказаний получается по следующей формуле:
$a = X \times w$.

Предположим, что Y - это обратимая матрица. Тогда заменим в формуле X на XY и воспользуемся уравнением метода наименьших квадратов: $w = (X^TX)X^Ty$.

$a = XY((XY)^TXY)^{-1}(XY)^Ty=XY(Y^TX^TXY)^{-1}Y^TX^Ty=XYY^{-1}(X^TX)^{-1}Y^{-T}Y^TX^Ty=XE(X^TX)^{-1}EX^Ty = X(X^TX)^{-1}X^Ty = X  w$

Единственное правило, которое нужно знать, чтобы вывести данное соотношение, это то, что если несколько матриц находится под одной операцией в скобках, то выносятся они в обратном порядке (в за рубежной литературе данное правило называется LIVE EVIL rule).

**Ответ**: не изменится.

## Алгоритм преобразования данных.

Учитывая выведенное соотношение в предыдущем разделе, очевидным алгоритмом будет умножение матрицы признаков на обратимую матрицу, которую создадим через np.random.normal.

Напишем функцию, которая примет наши изначальные признаки и вернет преобразованные.

In [2]:
def features_converter(X_original):
    X_original = X_original.values
    Y = np.random.normal(size=(len(X_original[1,:]),len(X_original[1,:])))
    print(Y.dot(np.linalg.inv(Y)))
    return X_original @ Y

<font color='blue'> 
### <u>КОММЕНТАРИЙ РЕВЬЮЕРА</u>
</font>
<font color='orange'>
Тут важно проверять обратимость матрицы!<br>
Напрмиер, проверить во это свойчство: P.dot(np.linalg.inv(P)) = единичная матрица.

Сначала обучим Линейную Регрессию на исходном датасете и посмотрим значение R2. Поскольку нам неважно качество модели, то и разбивать данные на обучающую и проверочную выборки не будем.

In [3]:
features = insurance_df.drop('Страховые выплаты',axis=1)
target   = insurance_df['Страховые выплаты']

model = LinearRegression()
model.fit(features,target)
predictions = model.predict(features)
r2 = r2_score(target,predictions)
print('Для исходных признаков R2 = {:.4f}'.format(r2))

Для исходных признаков R2 = 0.4249


Теперь воспользуемся нашим преобразователем.

In [4]:
new_features = features_converter(features)

model = LinearRegression()
model.fit(new_features,target)
predictions = model.predict(new_features)
r2 = r2_score(target,predictions)
print('Для преобразованных признаков R2 = {:.4f}'.format(r2))

[[ 1.00000000e+00  0.00000000e+00  4.44089210e-16  1.11022302e-16]
 [-3.05311332e-16  1.00000000e+00  0.00000000e+00  0.00000000e+00]
 [ 7.28583860e-17  6.93889390e-17  1.00000000e+00 -5.55111512e-17]
 [ 1.38777878e-17  6.93889390e-18  0.00000000e+00  1.00000000e+00]]
Для преобразованных признаков R2 = 0.4249


Результат идентичный, хоть и визуально матрицы признаков кажутся абсолютно разными. 

Получили подтверждение, что созданная рандомна матрица действительно обратима.

<font color='blue'> 
### <u>КОММЕНТАРИЙ РЕВЬЮЕРА</u>
</font>
<font color='green'>
Константин, лаконичная но верная по результату работа)<br>
Всё по делу, со смыслом. Теория и её реализация - в точку. Уместный анализ данных в самом начале.<br>

Пару маленьких ньюансов, всё же надо скорректировать(((<br>
1) При создании рандомной матрицы, надо проверить её обратимость. Т.е. получить единичную матрицу)<br>
2) По корреляциям: лучше такой вывод не оставлять. Или применить другую технику для проверки влияния фаткора на целевую переменную или убрать из проекта. А то не исправленная ошибка так и потянется......<font color='red'>
Скорректируй, пожалуйста)<br><font color='green'><font color='green'>
Спасибо) Буду ждать корректировок.

## Вывод

В данном проекте исходные данные нам достались чистые. Никаких особенных телодвижений делать не пришлось. Основной работой было доказательство неизменности предсказаний Линейной Регрессии при умножении нашей матрицы признаков на любую обратимую матрицу. Что мы и сделали.

По полученному выводу создали простейший алгоритм преобразования данных и, обучив нашу модель на исходном и преобразованном массиве данных, получили неизменное значение метрики.

Видимо любая обратимая матрица не искажает исходные данные.